In [ ]:
# !pip install python-docx

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3 as sql
import docx
import re
import datetime
import os
os.listdir()

['.DS_Store',
 '2nd.resume.ipynb',
 'resume.db',
 '경력기술서_예.docx',
 '경력기술서_강종구.docx',
 'resume.db-journal',
 '.ipynb_checkpoints']

In [2]:
now = str(datetime.datetime.now())
appl_id = now.split(' ')[0][2:].replace('-','')
appl_id

'200118'

In [3]:
def raw_data_parser(raw_data):
    for index, cont in enumerate(raw_data):
        sub_title = cont[0]
        if str(sub_title)[:4] == '지원사항':
            print('지원사항',index)
            index_a = index
        elif str(sub_title)[:4] == '학력사항':
            print('학력사항',index)
            index_b = index
        elif str(sub_title)[:4] == '경력사항':
            print('경력사항',index)
            index_c = index
        elif str(sub_title)[:4] == '지원동기':
            print('지원동기',index)
            index_d = index
        elif str(sub_title)[:5] == '나의 강점':
            print('나의 강점',index)
            index_e = index
        elif str(sub_title)[:5] == '지원 분야':
            print('지원 분야',index)
            index_f = index
    raw_who = raw_data[index_a : index_b]
    raw_edu = raw_data[index_b : index_c]
    raw_exp = raw_data[index_c : index_d]
    raw_mot = raw_data[index_d : index_e]
    raw_str = raw_data[index_e : index_f]
    raw_pro = raw_data[index_f :]
    now = str(datetime.datetime.now())
    appl_id = now.split(' ')[0][2:].replace('-','')
    
    return [raw_who, raw_edu, raw_exp, raw_mot, raw_str, raw_pro, appl_id]


def applicant_parser(aa, appl_id):
    dept = remove_space(aa[1][1])
    posi = remove_space(aa[1][3])
    name = remove_space(aa[3][1])
    dofb = remove_space(aa[3][3])
    gend = remove_space(aa[4][1])
    phon = remove_space(aa[4][3])
    mail = remove_space(aa[5][2])
    addr = remove_space(aa[6][2])
    port = remove_space(aa[7][2])
    
    addr = re.sub('[\n\xa0]', '', str(addr))

    return appl_id, dept, posi, name, dofb, gend, phon, mail, addr, port



def education_parser(bb, appl_id):
    edu_lst = []
    edu_size = len(bb)-1

    for i in range(1, edu_size + 1):
        if i == 0:
            pass
        else:
            if bb[i][0] == '고등학교':
                degr = bb[i][0]
                begi = remove_space(str(bb[i][1]).split('~')[0])
                endd = remove_space(str(bb[i][1]).split('~')[1])
                asso = bb[i][2]
                loca = bb[i][3]
                majo = bb[i][5]
                gpa1 = ' '
                gpa2 = ' '
                done = bb[i][7]
                edu_lst.append([appl_id, degr, begi, endd, asso, loca, majo, gpa1, gpa2, done])
            else:
                degr = bb[i][0]
                begi = remove_space(str(bb[i][1]).split('~')[0])
                endd = remove_space(str(bb[i][1]).split('~')[1])
                asso = bb[i][2]
                loca = bb[i][3]
                majo = bb[i][5]
                gpa1 = remove_space(str(bb[i][6]).split('/')[0])
                gpa2 = remove_space(str(bb[i][6]).split('/')[-1])
                done = bb[i][7]
                edu_lst.append([appl_id, degr, begi, endd, asso, loca, majo, gpa1, gpa2, done])

    return edu_lst


def work_index_parser(aa):
    index_lst = []
    cal_lst = []
    for i, j in enumerate(aa):
        if '경력 'in j[0]:
            index_lst.append(i)
        elif '희망' in j[0]:
            index_lst.append(i-1)
            
    for i, j in zip(index_lst, index_lst[1:]):
        cal_lst.append([i, j-1])
        
    return cal_lst


def work_parser(aa, appl_id):
    work_index = work_index_parser(aa)
    work_lst = []
    for i in range(0, len(work_index)):
        ind = work_index[i][0]

        comp = aa[ind][2]
        begi = remove_space(str(aa[ind][1]).split('~')[0])
        till = remove_space(str(aa[ind][1]).split('~')[-1])
        loca = aa[ind][3]
        dept = aa[ind][4]
        posi = aa[ind+1][1]
        duty = aa[ind+1][2]
        leav = aa[ind+1][3]
        sala = remove_space(str(aa[ind+1][4]).split('(')[0])
        det1 = aa[ind+2][1]
        det2 = aa[ind+2][3]
        mai1 = aa[ind+3][1]
        mai2 = aa[ind+3][3]
        work_lst.append([appl_id, comp, begi,till, loca, dept, posi, duty, leav, sala, det1, det2, mai1, mai2])
    return work_lst


def para_parser(aa, appl_id):
    a = re.sub('[\n\xa0]', '', str(aa[1][0]))
    return appl_id, a


def remove_space(aa):
    aa = re.sub('[ ]','',str(aa))
    return aa

In [4]:
# table check
def check_tables():
    query = "SELECT count(name) FROM sqlite_master WHERE type = 'table'"
    do.execute(query)
    print(do.fetchone())

In [5]:
doc = docx.Document('경력기술서_강종구.docx')
tables = doc.tables
table_size = len(tables)


raw_data = []
for table_num in range(0, table_size):
    for i, row in enumerate(tables[table_num].rows):
        text = [cell.text for cell in row.cells]
    
        row_data = tuple(text)#dict(zip(keys, text))
        raw_data.append(row_data)   
raw_data

[('지원사항', '지원사항', '지원사항', '지원사항'),
 ('지원부서', '인사부', '지원분야', '인사 데이터 분석'),
 ('기본인적사항', '기본인적사항', '기본인적사항', '기본인적사항'),
 ('성명', '강종구', '생년월일', '1987/06/17'),
 ('성별', '남', '휴대전화', '010-5502-2179'),
 ('이메일',
  'jay.jonggoo.kang@gmail.com',
  'jay.jonggoo.kang@gmail.com',
  'jay.jonggoo.kang@gmail.com'),
 ('주소',
  '서울시 강남구 테헤란로 108길 42 (대치동, MDM타워)\xa0',
  '서울시 강남구 테헤란로 108길 42 (대치동, MDM타워)\xa0',
  '서울시 강남구 테헤란로 108길 42 (대치동, MDM타워)\xa0'),
 ('포트폴리오',
  'https://github.com/jaygkay',
  'https://github.com/jaygkay',
  'https://github.com/jaygkay'),
 ('학력사항',
  '학력사항',
  '학력사항',
  '학력사항',
  '(해당 학력사항 없을 시 셀을 삭제 후 제출)',
  '(해당 학력사항 없을 시 셀을 삭제 후 제출)',
  '(해당 학력사항 없을 시 셀을 삭제 후 제출)',
  '(해당 학력사항 없을 시 셀을 삭제 후 제출)'),
 ('석사',
  '2018/12 ~ 2016/09',
  'DePaul University',
  'IL, USA',
  'IL, USA',
  '데이터\n사이언스',
  '3.0 / 4.0',
  '졸업'),
 ('학사 \n(편입 시)',
  '2015/12 ~ 2012/09',
  'University of Minnesota',
  'MN, USA',
  'MN, USA',
  '수학과\n경제학과',
  '2.0 / 4.0',
  '졸업'),
 ('학사',
  '2011/01 ~ 2012/05',
  'Un

In [6]:
raw_data_parser = raw_data_parser(raw_data)
who = applicant_parser(raw_data_parser[0], raw_data_parser[-1])
edu = education_parser(raw_data_parser[1], raw_data_parser[-1])
work = work_parser(raw_data_parser[2], raw_data_parser[-1])
motive = para_parser(raw_data_parser[3], raw_data_parser[-1])
strength = para_parser(raw_data_parser[4], raw_data_parser[-1])
project = para_parser(raw_data_parser[5], raw_data_parser[-1])


지원사항 0
학력사항 8
경력사항 13
지원동기 24
나의 강점 26
지원 분야 28


In [7]:
database_name = 'resume.db'
conn = sql.connect(database_name)
do = conn.cursor()

In [8]:
who

('200118',
 '인사부',
 '인사데이터분석',
 '강종구',
 '1987/06/17',
 '남',
 '010-5502-2179',
 'jay.jonggoo.kang@gmail.com',
 '서울시강남구테헤란로108길42(대치동,MDM타워)',
 'https://github.com/jaygkay')

In [9]:
query = "DROP TABLE IF EXISTS edu_query"
do.execute(query)

# who_table기존 값에 + edu_Id(1,2,3) + work_ID(1,2) + motive_Id + strenth_ID + project + ID

In [10]:
who_query = "CREATE TABLE IF NOT EXISTS who_table\
        (appl_id       INTEGER PRIMARY_KEY NOT_NULL,\
         department    VARCHAR(20),\
         position      VARCHAR(30),\
         name          VARCHAR(30),\
         dofb          DATETIME,\
         gender        VARCHAR(10),\
         phone         VARCHAR(20),\
         email         VARCHAR(50),\
         address       VARCHAR(80),\
         portfolio     VARCHAR(50))\
         "
do.execute(who_query)


edu_query = "CREATE TABLE IF NOT EXISTS education_table\
        (appl_id       INTEGER FOREIGN_KEY NOT_NULL,\
         degree       VARCHAR(20),\
         edu_start    VARCHAR(20),\
         edu_end      VARCHAR(20),\
         edu_name     VARCHAR(30),\
         edu_loc      VARCHAR(30),\
         edu_major    VARCHAR(30),\
         edu_gpa1     VARCHAR(20),\
         edu_gpa2     VARCHAR(20),\
         edu_status   VARCHAR(20))\
         "
do.execute(edu_query)

work_query = "CREATE TABLE IF NOT EXISTS work_table\
            (appl_id       INTEGER FOREIGN_KEY NOT_NULL,\
             company       VARCHAR(30),\
             work_start    VARCAHR(20),\
             work_end      VARCHAR(20),\
             work_loc      VARCHAR(30),\
             work_dept     VARCHAR(30),\
             work_title    VARCHAR(30),\
             work_duty     VARCAHR(50),\
             work_leave    VARCHAR(80),\
             work_salary   VARCHAR(20),\
             work_detail1  VARCHAR(50),\
             work_detail2  VARCHAR(50),\
             work_main1    VARCHAR(50),\
             work_main2    VARCHAR(50))"
do.execute(work_query)

motive_query = "CREATE TABLE IF NOT EXISTS motive_table\
              (appl_id       INTEGER FOREIGN_KEY NOT_NULL,\
               motive   VARCHAR(200))"
do.execute(motive_query)


strength_query = "CREATE TABLE IF NOT EXISTS strength_table\
              (appl_id       INTEGER FOREIGN_KEY NOT_NULL,\
               strength   VARCHAR(200))"
do.execute(strength_query)


project_query = "CREATE TABLE IF NOT EXISTS project_table\
              (appl_id       INTEGER FOREIGN_KEY NOT_NULL,\
               project   VARCHAR(200))"
do.execute(project_query)

do.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
for i in do:
    print(i)

('who_table',)
('education_table',)
('work_table',)
('motive_table',)
('strength_table',)
('project_table',)


In [14]:
who

('200118',
 '인사부',
 '인사데이터분석',
 '강종구',
 '1987/06/17',
 '남',
 '010-5502-2179',
 'jay.jonggoo.kang@gmail.com',
 '서울시강남구테헤란로108길42(대치동,MDM타워)',
 'https://github.com/jaygkay')

In [ ]:
len(who)

In [ ]:
who_insert = "INSERT INTO who_table VALUES (?,?,?,?,?,?,?,?,?,?)"
do.execute(who_insert, who)

In [13]:
query = "SELECT * FROM who_table"
do.execute(query)
do.fetchall()

[]

In [ ]:
len(edu), len(edu[0])

In [ ]:
for i in range(0, len(edu)):
    edu_insert = "INSERT INTO education_table VALUES (?,?,?,?,?,?,?,?,?,?)"
    do.execute(edu_insert, edu[i])

In [ ]:
query = "SELECT * FROM education_table"
do.execute(query)
do.fetchall()

In [ ]:
len(work), len(work[0])

In [ ]:
for i in range(0, len(work)):
    work_insert = "INSERT INTO work_table VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
    do.execute(work_insert, work[i])

In [ ]:
query = "SELECT * FROM work_table"
do.execute(query)
do.fetchall()

In [ ]:
motive_insert = "INSERT INTO motive_table VALUES (?,?)"
do.execute(motive_insert, motive)

In [ ]:
query = "SELECT * FROM motive_table"
do.execute(query)
do.fetchall()

In [ ]:
strength_insert = "INSERT INTO strength_table VALUES (?,?)"
do.execute(strength_insert, strength)

In [ ]:
query = "SELECT * FROM strength_table"
do.execute(query)
do.fetchall()

In [ ]:
project_insert = "INSERT INTO project_table VALUES (?,?)"
do.execute(project_insert, project)

In [ ]:
query = "SELECT * FROM project_table"
do.execute(query)
do.fetchall()

In [ ]:
query = '''SELECT * 
         FROM who_table as a
         LEFT JOIN education_table as b
              
         ON a.appl_id = b.appl_id
            '''
    

do.execute(query)
do.fetchall()

In [ ]:
query = "SELECT * from who_table as a\
         INNER JOIN education_table as b on a.appl_id = b.appl_id\
         INNER JOIN work_table as c on b.appl_id = c.appl_id\
         INNER JOIN motive_table as d on c.appl_id = d.appl_id\
         INNER JOIN strength_table as e on d.appl_id = e.appl_id\
         INNER JOIN project_table as f on e.appl_id = f.appl_id"

do.execute(query)
do.fetchall()